# **C.O.L.L.A.B.**
<hr style="border:2px solid black">

## Goal: 
* 
* 
<hr style="border:2px solid black">

# Imports

In [125]:
import pandas as pd
import numpy as np
from wrangle import wrangle
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve

<hr style="border:2px solid black">

# **Acquire**
Plan --> **Acquire** --> Prepare --> Explore --> Model --> Deliver

* 

In [126]:
    # Acquire kaggle data
    col = pd.read_csv('cost_of_living_msa.csv')

    # Acquire Census Data
    family_income = pd.read_csv('B19126_msa_2022.csv')
    family_count = pd.read_csv('DP02_msa_2022.csv')

In [127]:
    # grab the indices 6 and 7 from the dataset and store as male
    male = family_count[6:8]
    # grab the indices 10 and 11 from the dataset and store as female
    female = family_count[10:12]
    # join the two dataframes
    family_count = pd.concat([female, male], ignore_index=True, axis = 0)
    # since we have unnecessary columns in between the ones we want, let's skip every other column
    columns = family_count.columns[2::2]
    # now let's drop every other column
    family_count.drop(columns, axis = 1, inplace = True)
    # for legibility, let's rename the index into named columns
    family_count = family_count.T.rename(columns = {0:'fem_total(count)', 1:'fem_with_kid(count)', 2:'male_total(count)', 3:'male_with_kid(count)'})
    # we still have a row that is full of the old title, let's remove that
    family_count = family_count.drop(family_count.index[0])

    # now we can remove null values and remove commas from our numbers
    family_count = family_count.fillna('0')
    family_count['fem_total(count)'] = family_count['fem_total(count)'].str.replace(',', '').astype(int)
    family_count['fem_with_kid(count)'] = family_count['fem_with_kid(count)'].str.replace(',', '').astype(int)
    family_count['male_total(count)'] = family_count['male_total(count)'].str.replace(',', '').astype(int)
    family_count['male_with_kid(count)'] = family_count['male_with_kid(count)'].str.replace(',', '').astype(int)

    # transform strings into floats
    family_count['fem_no_kid(count)'] = family_count['fem_total(count)'] - family_count['fem_with_kid(count)']
    family_count['male_no_kid(count)'] = family_count['male_total(count)'] - family_count['male_with_kid(count)']

    # grab the associated indices for male, female, and total
    tot = family_income[4:6]
    male = family_income[10:12]
    female = family_income[14:16]
    # bring those together
    family_income = pd.concat([tot, female, male], ignore_index=True, axis = 0)
    # for legibility, let's rename the index into named columns
    family_income = family_income.T.rename(columns = {0:'mar_with_kid', 1:'mar_no_kid', 2:'fem_with_kid(income)', 
                                                      3:'fem_no_kid(income)', 4:'male_with_kid(income)', 5:'male_no_kid(income)'})
    family_income = family_income.drop(family_income.index[0])

    # let's remove special characters from our strings
    family_income['mar_with_kid'] = family_income['mar_with_kid'].str.replace('+', '')
    family_income['male_no_kid(income)'] = family_income['male_no_kid(income)'].str.replace('+', '').replace('-', '')
    family_income['male_with_kid(income)'] = family_income['male_no_kid(income)'].str.replace('+', '').replace('-', '')

    # now let's remove all other special characters and fill some of our blank values with 0 to be dropped later
    family_income['mar_with_kid'] = family_income['mar_with_kid'].replace('', '0').str.replace(',', '').astype(int)
    family_income['mar_no_kid'] = family_income['mar_no_kid'].replace('', '0').str.replace(',', '').astype(int)
    family_income['fem_no_kid(income)'] = family_income['fem_no_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
    family_income['fem_with_kid(income)'] = family_income['fem_with_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
    family_income['male_no_kid(income)'] = family_income['male_no_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
    family_income['male_with_kid(income)'] = family_income['male_with_kid(income)'].replace('', '0').str.replace(',', '').astype(int)

    # finally let's bring these two together
    family = pd.concat([family_income, family_count], axis = 1)
    
    # make the column names of col lowercase
    col.columns = col.columns.str.lower()
    col.rename(columns = {'areaname' :'msa'}, inplace=True)

In [128]:
    # Calculate income for each family type
    family['single_no_kid(income)'] = (family['fem_no_kid(income)'] * family['fem_no_kid(count)'] + family['male_no_kid(income)'] * family['male_no_kid(count)'])/(family['fem_no_kid(count)'] + family['male_no_kid(count)'])
    family['single_with_kid'] = ((family['fem_with_kid(income)'] * family['fem_with_kid(count)']) + (family['male_with_kid(income)'] * family['male_with_kid(count)']))/(family['fem_with_kid(count)'] + family['male_with_kid(count)'])
    # Drop unnecessary columns from family
    family = family[['mar_with_kid', 'mar_no_kid', 'single_no_kid(income)', 'single_with_kid']]
    # replace NaN with 0
    family = family.fillna(0)
    # Round values to the nearest integer and convert to int
    family['mar_with_kid'] = round(family['mar_with_kid']).astype(int)
    family['mar_no_kid'] = round(family['mar_no_kid']).astype(int)
    family['single_no_kid(income)'] = round(family['single_no_kid(income)']).astype(int)
    family['single_with_kid'] = round(family['single_with_kid']).astype(int)

In [129]:
    family.reset_index(inplace = True)
    # rename the first column to 'county'
    family.rename(columns = {'index':'msa'}, inplace = True)

In [112]:
family.msa

0                       Abilene, TX Metro Area!!Estimate
1             Aguadilla-Isabela, PR Metro Area!!Estimate
2                         Akron, OH Metro Area!!Estimate
3                        Albany, GA Metro Area!!Estimate
4                Albany-Lebanon, OR Metro Area!!Estimate
                             ...                        
387                       Yauco, PR Metro Area!!Estimate
388                York-Hanover, PA Metro Area!!Estimate
389    Youngstown-Warren-Boardman, OH-PA Metro Area!!...
390                   Yuba City, CA Metro Area!!Estimate
391                        Yuma, AZ Metro Area!!Estimate
Name: msa, Length: 392, dtype: object

In [142]:
len(family)

392

In [141]:
col.iloc[::10].head(30)

,case_id,state abv.,msa,family,housing,food,transportation,healthcare,other necessities,childcare,taxes,total,median_family_income
0,3172,TX,"Abilene, TX MSA",1p0c,8484.0,3014.58108,10162.77900,5389.47984,4166.38956,0.0,4687.83576,35905.0668,66485.14729
10,3173,OH,"Akron, OH MSA",1p0c,6792.0,3236.27340,9402.31344,4309.35024,3633.63900,0.0,4404.13476,31777.7112,77541.41057
20,3174,GA,"Albany, GA MSA",1p0c,7488.0,3182.99892,10238.85504,5149.47984,3866.52360,0.0,5757.92760,35683.7844,55237.98817
30,3175,OR,"Albany, OR MSA",1p0c,9528.0,3247.41468,11009.33424,4604.37996,4629.03660,0.0,7442.64480,40460.8092,68382.50732
40,3176,NY,"Albany-Schenectady-Troy, NY MSA",1p0c,9708.0,3679.23228,9954.05784,5668.53996,4850.72208,0.0,6768.18756,40628.7396,95476.30918
50,3177,NM,"Albuquerque, NM MSA",1p0c,7452.0,3221.06796,9490.17036,3476.37996,3867.27360,0.0,4569.05088,32075.9412,69364.11018
60,3178,LA,"Alexandria, LA MSA",1p0c,6480.0,2978.13132,10728.57132,5545.47984,3427.05396,0.0,5142.59580,34301.8332,58290.55052
70,3179,PA,"Allentown-Bethlehem-Easton, PA HUD Metro FMR Area",1p0c,9456.0,3454.48164,9336.75444,4680.69612,4677.97704,0.0,5798.74032,37404.6504,81129.97748
80,3180,NJ,"Warren County, NJ HUD Metro FMR Area",1p0c,11256.0,3683.16984,10869.48048,4672.53996,5413.05024,0.0,6285.60132,42179.8416,103354.14880
90,3181,PA,"Altoona, PA MSA",1p0c,6888.0,3185.16324,10100.36940,3508.53996,3649.90428,0.0,4813.14840,32145.1260,66065.98776


In [104]:
    # grab the indices 6 and 7 from the dataset and store as male
    male = family_count[6:8]
    # grab the indices 10 and 11 from the dataset and store as female
    female = family_count[10:12]
    # join the two dataframes
    family_count = pd.concat([female, male], ignore_index=True, axis = 0)
    # since we have unnecessary columns in between the ones we want, let's skip every other column
    columns = family_count.columns[2::2]
    # now let's drop every other column
    family_count.drop(columns, axis = 1, inplace = True)
    # for legibility, let's rename the index into named columns
    family_count = family_count.T.rename(columns = {0:'fem_total(count)', 1:'fem_with_kid(count)', 2:'male_total(count)', 3:'male_with_kid(count)'})
    # we still have a row that is full of the old title, let's remove that
    family_count = family_count.drop(family_count.index[0])


In [105]:
family_count

,fem_total(count),fem_with_kid(count),male_total(count),male_with_kid(count)
fem_with_kid(count),5279,1138,17239,3524
male_with_kid(count),1760,548,4973,293
male_no_kid(count),17881,12645,74104,8892


In [106]:
family_count

,fem_total(count),fem_with_kid(count),male_total(count),male_with_kid(count)
fem_with_kid(count),5279,1138,17239,3524
male_with_kid(count),1760,548,4973,293
male_no_kid(count),17881,12645,74104,8892


In [107]:

    # now we can remove null values and remove commas from our numbers
    family_count = family_count.fillna('0')
    family_count['fem_total(count)'] = family_count['fem_total(count)'].str.replace(',', '').astype(int)
    family_count['fem_with_kid(count)'] = family_count['fem_with_kid(count)'].str.replace(',', '').astype(int)
    family_count['male_total(count)'] = family_count['male_total(count)'].str.replace(',', '').astype(int)
    family_count['male_with_kid(count)'] = family_count['male_with_kid(count)'].str.replace(',', '').astype(int)

    # transform strings into floats
    family_count['fem_no_kid(count)'] = family_count['fem_total(count)'] - family_count['fem_with_kid(count)']
    family_count['male_no_kid(count)'] = family_count['male_total(count)'] - family_count['male_with_kid(count)']

    # grab the associated indices for male, female, and total
    tot = family_income[4:6]
    male = family_income[10:12]
    female = family_income[14:16]
    # bring those together
    family_income = pd.concat([tot, female, male], ignore_index=True, axis = 0)
    # for legibility, let's rename the index into named columns
    family_income = family_income.T.rename(columns = {0:'mar_with_kid', 1:'mar_no_kid', 2:'fem_with_kid(income)', 
                                                      3:'fem_no_kid(income)', 4:'male_with_kid(income)', 5:'male_no_kid(income)'})
    family_income = family_income.drop(family_income.index[0])

    # let's remove special characters from our strings
    family_income['mar_with_kid'] = family_income['mar_with_kid'].str.replace('+', '')
    family_income['male_no_kid(income)'] = family_income['male_no_kid(income)'].str.replace('+', '').replace('-', '')
    family_income['male_with_kid(income)'] = family_income['male_no_kid(income)'].str.replace('+', '').replace('-', '')

    # now let's remove all other special characters and fill some of our blank values with 0 to be dropped later
    family_income['mar_with_kid'] = family_income['mar_with_kid'].replace('', '0').str.replace(',', '').astype(int)
    family_income['mar_no_kid'] = family_income['mar_no_kid'].replace('', '0').str.replace(',', '').astype(int)
    family_income['fem_no_kid(income)'] = family_income['fem_no_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
    family_income['fem_with_kid(income)'] = family_income['fem_with_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
    family_income['male_no_kid(income)'] = family_income['male_no_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
    family_income['male_with_kid(income)'] = family_income['male_with_kid(income)'].replace('', '0').str.replace(',', '').astype(int)

    # finally let's bring these two together
    family = pd.concat([family_income, family_count], axis = 1)

AttributeError: Can only use .str accessor with string values!

In [ ]:
col = pd.read_csv('cost_of_living_msa.csv')

In [ ]:
family_income = pd.read_csv('B19126.csv')

In [ ]:
family_income # county

,Label (Grouping),"Baldwin County, Alabama!!Estimate","Calhoun County, Alabama!!Estimate","Cullman County, Alabama!!Estimate","DeKalb County, Alabama!!Estimate","Elmore County, Alabama!!Estimate","Etowah County, Alabama!!Estimate","Houston County, Alabama!!Estimate","Jefferson County, Alabama!!Estimate","Lauderdale County, Alabama!!Estimate",...,"Bayamón Municipio, Puerto Rico!!Estimate","Caguas Municipio, Puerto Rico!!Estimate","Carolina Municipio, Puerto Rico!!Estimate","Guaynabo Municipio, Puerto Rico!!Estimate","Mayagüez Municipio, Puerto Rico!!Estimate","Ponce Municipio, Puerto Rico!!Estimate","San Juan Municipio, Puerto Rico!!Estimate","Toa Alta Municipio, Puerto Rico!!Estimate","Toa Baja Municipio, Puerto Rico!!Estimate","Trujillo Alto Municipio, Puerto Rico!!Estimate"
0,Median family income in the past 12 months (in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total:,"83,900","69,301","75,394","54,784","79,378","68,590","75,042","81,880","74,956",...,"32,687","33,080","40,273","56,380","25,030","21,488","35,044","48,233","35,528","45,742"
2,Married-couple family --,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Total,"96,333","82,338","86,281","72,883","97,421","79,407","96,500","110,937","82,955",...,"42,416","43,882","50,808","74,696","35,147","30,065","59,356","62,018","45,896","68,617"
4,With own children of the house...,"106,141","69,568","85,096","57,004","95,917","92,321","108,097","129,382","91,553",...,"57,523","63,007","51,731","89,709","43,765","37,900","76,165","48,214","65,154","60,125"
5,No own children of the househo...,"91,016","86,169","88,386","75,605","101,406","73,509","91,675","98,502","80,758",...,"36,948","36,787","50,127","72,339","33,437","27,092","52,769","64,840","43,236","70,569"
6,Other family --,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Total,"42,456","39,889","36,370","46,523","46,450","35,339","40,830","41,207","52,742",...,"24,044","21,274","27,047","30,630","11,818","14,306","21,409","32,537","27,806","26,772"
8,"Male householder, no spouse pr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Total,"48,635","72,555","52,153","52,711","74,049","55,827","50,385","54,921","76,569",...,"29,372","19,600","31,476","50,620","22,658","22,480","30,721","54,026","28,788","26,445"


In [ ]:
family_income # msa

,Label (Grouping),"Abilene, TX Metro Area!!Estimate","Abilene, TX Metro Area!!Margin of Error","Aguadilla-Isabela, PR Metro Area!!Estimate","Aguadilla-Isabela, PR Metro Area!!Margin of Error","Akron, OH Metro Area!!Estimate","Akron, OH Metro Area!!Margin of Error","Albany, GA Metro Area!!Estimate","Albany, GA Metro Area!!Margin of Error","Albany-Lebanon, OR Metro Area!!Estimate",...,"Yauco, PR Metro Area!!Estimate","Yauco, PR Metro Area!!Margin of Error","York-Hanover, PA Metro Area!!Estimate","York-Hanover, PA Metro Area!!Margin of Error","Youngstown-Warren-Boardman, OH-PA Metro Area!!Estimate","Youngstown-Warren-Boardman, OH-PA Metro Area!!Margin of Error","Yuba City, CA Metro Area!!Estimate","Yuba City, CA Metro Area!!Margin of Error","Yuma, AZ Metro Area!!Estimate","Yuma, AZ Metro Area!!Margin of Error"
0,Median family income in the past 12 months (in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total:,"68,388","±2,697","20,420",±736,"76,842","±1,054","56,166","±1,816","68,860",...,"20,194","±1,030","82,154","±1,323","63,220","±1,020","69,812","±3,334","54,897","±1,983"
2,Married-couple family --,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Total,"79,677","±2,504","26,369",±906,"91,608","±1,490","78,304","±2,045","78,115",...,"24,838","±1,472","93,649","±1,310","76,853","±1,172","84,677","±3,770","61,605","±1,931"
4,With own children of the house...,"80,816","±4,263","35,292","±2,933","105,659","±3,612","81,031","±7,561","75,538",...,"29,661","±8,278","102,849","±2,404","87,669","±2,506","87,642","±7,232","69,724","±5,715"
5,No own children of the househo...,"78,834","±3,089","23,907","±1,033","85,640","±1,631","77,308","±3,284","79,222",...,"23,136","±1,447","88,604","±2,226","71,927","±1,130","82,495","±5,376","58,769","±2,253"
6,Other family --,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Total,"38,001","±2,449","13,046",±666,"40,325","±1,554","28,803","±2,185","44,530",...,"12,959","±1,489","50,235","±1,691","32,910","±1,443","44,506","±2,483","32,421","±2,141"
8,"Male householder, no spouse pr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Total,"42,540","±6,836","14,543","±1,167","53,203","±2,551","40,318","±7,238","56,202",...,"19,238","±3,717","61,736","±2,927","46,347","±3,784","52,135","±3,265","46,702","±10,248"


In [ ]:
family_count = pd.read_csv('DP02_msa.csv')

In [ ]:
family_count

,Label (Grouping),"Abilene, TX Metro Area!!Estimate","Abilene, TX Metro Area!!Percent","Aguadilla-Isabela, PR Metro Area!!Estimate","Aguadilla-Isabela, PR Metro Area!!Percent","Akron, OH Metro Area!!Estimate","Akron, OH Metro Area!!Percent","Albany, GA Metro Area!!Estimate","Albany, GA Metro Area!!Percent","Albany-Lebanon, OR Metro Area!!Estimate",...,"Yauco, PR Metro Area!!Estimate","Yauco, PR Metro Area!!Percent","York-Hanover, PA Metro Area!!Estimate","York-Hanover, PA Metro Area!!Percent","Youngstown-Warren-Boardman, OH-PA Metro Area!!Estimate","Youngstown-Warren-Boardman, OH-PA Metro Area!!Percent","Yuba City, CA Metro Area!!Estimate","Yuba City, CA Metro Area!!Percent","Yuma, AZ Metro Area!!Estimate","Yuma, AZ Metro Area!!Percent"
0,HOUSEHOLDS BY TYPE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total households,"64,331","64,331",NaN,NaN,"291,655","291,655","57,104","57,104","49,113",...,NaN,NaN,"176,428","176,428","228,080","228,080","59,591","59,591","72,716","72,716"
2,Married-couple household,"31,958",49.7%,NaN,NaN,"128,274",44.0%,"22,391",39.2%,"25,207",...,NaN,NaN,"92,818",52.6%,"97,858",42.9%,"29,726",49.9%,"37,785",52.0%
3,With children of the householder u...,"12,262",19.1%,NaN,NaN,"45,509",15.6%,"7,576",13.3%,"8,694",...,NaN,NaN,"33,867",19.2%,"30,239",13.3%,"13,938",23.4%,"14,477",19.9%
4,Cohabiting couple household,"4,747",7.4%,NaN,NaN,"21,458",7.4%,"3,161",5.5%,"3,635",...,NaN,NaN,"14,569",8.3%,"15,875",7.0%,"4,960",8.3%,"4,871",6.7%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,West Indian (excluding Hispanic origin...,561,0.3%,NaN,NaN,"1,374",0.2%,814,0.5%,48,...,NaN,NaN,"1,122",0.2%,765,0.1%,443,0.2%,243,0.1%
168,COMPUTERS AND INTERNET USE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,Total households,"64,331","64,331",NaN,NaN,"291,655","291,655","57,104","57,104","49,113",...,NaN,NaN,"176,428","176,428","228,080","228,080","59,591","59,591","72,716","72,716"
170,With a computer,"59,113",91.9%,NaN,NaN,"269,864",92.5%,"51,458",90.1%,"46,342",...,NaN,NaN,"161,964",91.8%,"201,029",88.1%,"55,024",92.3%,"65,269",89.8%


<hr style="border:2px solid black">

# **Prepare**

**Prepare Actions:**
* To better estimate family income based on family composition, use the estimates of family count by family type(Married, Single Male with kids, Single Male without kids, Single Female with kids, Single Female without kids) and the estimates of income by family type. e.g. (female_with_kids (income) * female_with_kids (count) + (male_with_kids (income) * male_with_kids (count)))/2
* Merge transformed census dataframe (family) on to the cost_of_living dataframe (col) on county.
* Adjust for inflation 2020 to 2022

In [ ]:
cost, dash = wrangle()
cost

,county,parents,children,housing,food,transportation,healthcare,other,childcare,taxes,total,median_income
0,baldwin_county_al,1,0,9390,4048,11695,6096,4869,0,7035,43133,68231
1,baldwin_county_al,1,1,12058,5966,12558,9202,6530,6498,7734,60546,40290
2,baldwin_county_al,1,2,12058,8740,14025,12309,7536,16726,10329,81723,40290
3,baldwin_county_al,1,3,16334,11660,14661,15415,10143,19874,14165,102253,40290
4,baldwin_county_al,1,4,16334,14273,15010,18522,11090,19874,14640,109742,40290
...,...,...,...,...,...,...,...,...,...,...,...,...
8239,natrona_county_wy,2,0,9638,6488,14377,15581,5843,0,6990,58917,94332
8240,natrona_county_wy,2,1,11351,8075,15391,19471,7039,10372,8267,79966,103027
8241,natrona_county_wy,2,2,11351,10227,16148,23361,7819,22325,9462,100693,103027
8242,natrona_county_wy,2,3,16242,12487,16629,27251,10410,26003,11935,120957,103027


In [ ]:
cost['affordability_ratio'] = round(cost.median_income / cost.total, 2)
cost

,county,parents,children,housing,food,transportation,healthcare,other,childcare,taxes,total,median_income,affordability_ratio
0,baldwin_county_al,1,0,9390,4048,11695,6096,4869,0,7035,43133,68231,1.58
1,baldwin_county_al,1,1,12058,5966,12558,9202,6530,6498,7734,60546,40290,0.67
2,baldwin_county_al,1,2,12058,8740,14025,12309,7536,16726,10329,81723,40290,0.49
3,baldwin_county_al,1,3,16334,11660,14661,15415,10143,19874,14165,102253,40290,0.39
4,baldwin_county_al,1,4,16334,14273,15010,18522,11090,19874,14640,109742,40290,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8239,natrona_county_wy,2,0,9638,6488,14377,15581,5843,0,6990,58917,94332,1.60
8240,natrona_county_wy,2,1,11351,8075,15391,19471,7039,10372,8267,79966,103027,1.29
8241,natrona_county_wy,2,2,11351,10227,16148,23361,7819,22325,9462,100693,103027,1.02
8242,natrona_county_wy,2,3,16242,12487,16629,27251,10410,26003,11935,120957,103027,0.85


<hr style="border:2px solid black">

# **Explore**
Plan --> Acquire --> Prepare --> **Explore** --> Model --> Deliver

<hr style="border:2px solid black">

# **Modeling**
Plan --> Acquire --> Prepare --> Explore --> **Model** --> Deliver


<hr style="border:2px solid black">

# **Comparing Models**


<hr style="border:2px solid black">

## **Best Model Evaluation on Test**

### <u>Modeling Summary:</u>

>* 

<hr style="border:2px solid black">

# **Conclusions**


### <u>Modeling:</u>

>**<u>:</u>**
>* 

### <u>Recommendations:</u>
>* 

### <u>Next Steps:</u>
>* 